In [19]:
#import the files 
import re 
import nltk
import ssl
import pandas as pd 
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [21]:
cv = CountVectorizer()
train_data_2 = pd.read_csv('training_set.csv')
test_data = pd.read_csv('testing_set.csv')

X_train = cv.fit_transform(train_data_2["Review"]).toarray()
y_train = train_data_2["Liked"].values

X_test = cv.transform(test_data["Review"]).toarray()
y_test = test_data["Liked"].values

#train the model 
classifier = BernoulliNB(alpha=0.8)
classifier.fit(X_train, y_train)


# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print ("Confusion Matrix:\n",cm)

# Accuracy, Precision and Recall
score1 = accuracy_score(y_test,y_pred)

score2 = precision_score(y_test,y_pred)
score3= recall_score(y_test,y_pred)
print("\n")
print("Accuracy is ",round(score1*100,2),"%")
print("Precision is ",round(score2,2))
print("Recall is ",round(score3,2))

Confusion Matrix:
 [[114  21]
 [ 38 127]]


Accuracy is  80.33 %
Precision is  0.86
Recall is  0.77


In [24]:
len(y_pred)

300

In [171]:
business = pd.read_csv("cleaned_business.csv")
reviews = pd.read_csv("cleanish_reviews.csv") 
ids = pd.read_csv('list_buiss_id.txt', sep=" ", header=None)
ids.columns = ["id"]

In [172]:
columns = ["business_id", "business_name", "our_score", "stars"]
df = pd.DataFrame(columns=columns)
#usiness.info()

In [173]:
for i in range(len(ids)): 
    id_curr = ids.iloc[i].id
    
    df = df.append({"business_id": id_curr}, ignore_index=True)
    
    #Review data that matches the current business id 
    all_reviews = reviews[reviews["business_id"] == id_curr]
    
    #vectorize the reviews for that resturant 
    X_test = cv.transform(all_reviews["text"]).toarray()
    #creates the predicted sentiment for each review 
    y_pred = classifier.predict(X_test)
    #you dont have to round 
    rounded_y_pred = np.mean(y_pred)*100
    #update the column with our calculated score 
    business.loc[business['business_id'] == id_curr, "food_check_score"] = rounded_y_pred
    
    
    
    
df.head()

,business_id,business_name,our_score,stars
0,s2B_ZeSGTlQ3qbm0dOY5rg,NaN,NaN,NaN
1,tYYJYqbw1zasFUCqrkawKQ,NaN,NaN,NaN
2,wtDLILBaj5BGEbYvWGfNUw,NaN,NaN,NaN
3,Mnu3EPpuzu3UUN1PIwFRoQ,NaN,NaN,NaN
4,MrSHPvHVdE_VzVN7iRz45Q,NaN,NaN,NaN


In [174]:
list_id = ids['id']
type(list_id)

pandas.core.series.Series

In [175]:
#filter the business dataframe for only busineses in list 
list_id = ids['id']
filtered_business = business[business['business_id'].isin(list_id)]

In [176]:
business[business["business_id"] == id_curr]

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,food_check_score
2985,6xowh_GdOsRrz9QWN6nMxA,Uncle Tetsu's Japanese Angel Cafe,191 Dundas Street W,Toronto,ON,M5G 1C7,43.654923,-79.386795,3.5,168,1,"{'BikeParking': 'True', 'GoodForKids': 'True',...","Food, Patisserie/Cake Shop, Desserts, Restaura...","{'Monday': '11:0-20:0', 'Tuesday': '11:0-20:0'...",50.0


In [177]:
filtered_business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,food_check_score
174,yGMCl0vYigshkXiZFIDTNw,Mimi's Cafe,7450 W Bell Rd,Glendale,AZ,85308,33.638654,-112.219354,3.0,184,1,"{'RestaurantsDelivery': 'False', 'CoatCheck': ...","Nightlife, Restaurants, American (Traditional)...","{'Monday': '7:0-21:0', 'Tuesday': '7:0-21:0', ...",40.703518
1398,Mnu3EPpuzu3UUN1PIwFRoQ,Maui Pasta,"7704 E Doubletree Ranch Rd, Ste 115",Scottsdale,AZ,85258,33.565229,-111.915444,4.5,52,1,"{'RestaurantsGoodForGroups': 'True', 'Business...","Italian, Pasta Shops, Restaurants, Specialty F...","{'Monday': '0:0-0:0', 'Tuesday': '7:0-20:0', '...",82.692308
1554,qBRlwvEuAeTBh_yQLJ4ZVg,Bières et Compagnie,4350 Rue Saint-Denis,Montréal,QC,H2J 2L1,45.522839,-73.580762,3.5,51,0,"{'GoodForKids': 'False', 'RestaurantsGoodForGr...","Nightlife, Belgian, Beer, Wine & Spirits, Pubs...","{'Monday': '11:30-0:0', 'Tuesday': '11:30-0:0'...",69.230769
1777,l-D0uS-tgmhkFpHKBXvPQA,Chino Bandido,1825 W Chandler Blvd,Chandler,AZ,85224,33.304492,-111.873637,3.5,380,0,"{'Corkage': 'False', 'RestaurantsAttire': ""'ca...","Chinese, Restaurants, Asian Fusion, Mexican","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...",38.987342
1855,qsJkjQoM3Hdg6Gtqp2S9_w,Archi's Thai Cafe,9350 W Sahara Ave,Las Vegas,NV,89117,36.147153,-115.299976,3.5,562,1,"{'RestaurantsAttire': ""u'casual'"", 'Restaurant...","Thai, Restaurants, Food","{'Monday': '11:0-22:0', 'Wednesday': '11:0-22:...",55.536332
